In [1]:
import numpy as np
from numpy import linalg as LA
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal, fft
import math as math
import os
from matplotlib.patches import Polygon
import copy
from scipy.stats import norm
import statistics

In [2]:
import pickle
a_file = open("HCPS.pkl", "rb") #remember to close afterwards
HCPS = pickle.load(a_file)
a_file.close()

HCPS.keys()

dict_keys(['subject0', 'subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14'])

## Data analysis

In [3]:
# Data analysis
def FFT(data,N):
  return fft.fft(data)/N

def IFFT(data,N):
  return (fft.ifft(data)*N).real

M = lambda s : 1/(s**2 + s) #2nd order machine

num_cond = 7 #number of conditions
participants_num = len(HCPS.keys())

#scaling factors for output screen and input slider
scaleOutputScreen = 1/4
scaleInput = 0.04616974606700115

#number of trials for each condition
trialnum = [0] * num_cond 
for c in range(num_cond):
  trialnum[c] = len(HCPS['subject0']['condition'+str(c)].keys())         #number of data trials, trial0 ~ trial3

#parameters (same for all three conditions)
fs = 60                               #pygame update rate 60 Hz
base_freq = 0.05                      #1/20 Hz
N = len(HCPS['subject0']['condition0']['trial0']['time_'])    #data length
xf_all = fft.fftfreq(N, 1./ fs)       #freq (x-axis) both + and - terms
xf = fft.fftfreq(N, 1./ fs)[:N//2]    #freq (x-axis) positive-frequency terms
M_h = M(1.j*2*np.pi*xf_all)           #M_hat = 1/ ((jw)^2 + (jw))
t = HCPS['subject0']['condition0']['trial0']['time_']         #time
prime = np.asarray([2, 3, 5, 7, 
                    11, 13, 17, 19])  #prime numbers
stimulated_index = prime*2 #array([ 4,  6, 10, 14, 22, 26, 34, 38])
stimulated_freq = prime*base_freq

def Analysis(data,trialnum):
  r = []      #r, reference (time)
  Md = []     #Md, disturbance (time)
  y = []      #y, output (time)
  r_h = []    #r_hat, reference (freq)
  y_h = []    #y_hat, output (freq)
  Md_h = []   #Md_hat, disturbance (freq)
  Wr_h = []   #Wr_hat, M^{-1}r_hat (freq)
  Wr = []     #Wr, M^{-1}r (time)
  u_h = []    #u_hat, input (freq)
  u = []      #u, input (time)
  u0_h = []   #u0_hat, emg input (freq)
  u0 = []     #u0, emg input (time)
  u1_h = []   #u1_hat, slider input (freq)
  u1 = []     #u1, slider input (time)
  d_h = []    #d_hat, disturbance (freq)
  d = []      #d, disturbance (time)

  for i in range(trialnum):
    
    #(1) output 
    r.append( data['trial'+str(i)]['ref_'] )
    d.append( data['trial'+str(i)]['dis_'] )
    y.append( data['trial'+str(i)]['out_'] )
    
    #(2) fft (entire freq term)
    r_h.append( FFT(r[i],N) )
    y_h.append( FFT(y[i],N) )
    d_h.append( FFT(d[i],N) )

    #(3) M^{-1}r_hat = Wr_hat = r_hat / M_hat in freq domain and Wr in time domain
    Wr_h.append( r_h[i] / M_h )
    Wr_h[i][0] = 0 #nan
    Wr.append( IFFT(Wr_h[i],N) )
    
    #(4) input(u): u_hat in freq domain and u in time domain
    # Umean = np.mean(data['trial'+str(i)]['inp_'])
    u_h.append( FFT(data['trial'+str(i)]['inp_'],N) ) #combined input
    u.append( IFFT(u_h[i],N) )

    u0_h.append( FFT(data['trial'+str(i)]['inp0_'],N) ) #seperated emg input
    u0.append( IFFT(u0_h[i],N) )

    u1_h.append( FFT(data['trial'+str(i)]['inp1_'],N) ) #seperated slider input 
    u1.append( IFFT(u1_h[i],N) )

    #(5) disturbance(d): d_hat = Md_hat/M_hat in freq domain and d in time domain
    Md_h.append( d_h[i] * M_h )
    Md_h[i][0] = 0 #nan
    Md.append( IFFT(Md_h[i],N) )

    #output scaling
    r[i] = r[i]*scaleOutputScreen
    y[i] = y[i]*scaleOutputScreen
    Md[i] = Md[i]*scaleOutputScreen
    r_h[i] = r_h[i]*scaleOutputScreen
    y_h[i] = y_h[i]*scaleOutputScreen
    Md_h[i] = Md_h[i]*scaleOutputScreen

    #input scaling
    d[i] = d[i]*scaleInput
    Wr[i] = Wr[i]*scaleInput
    u[i] = u[i]*scaleInput
    u0[i] = u0[i]*scaleInput
    u1[i] = u1[i]*scaleInput
    d_h[i] = d_h[i]*scaleInput
    Wr_h[i] = Wr_h[i]*scaleInput
    u_h[i] = u_h[i]*scaleInput
    u0_h[i] = u0_h[i]*scaleInput
    u1_h[i] = u1_h[i]*scaleInput    

  #create dict
  time_domain = {'Md':Md,'r':r,'y':y,'d':d,'Wr':Wr,'u':u,'u0':u0,'u1':u1} #time domain
  freq_domain = {'MD':Md_h,'R':r_h,'Y':y_h,'D':d_h,'WR':Wr_h,'U':u_h,'U0':u0_h,'U1':u1_h} #freq domain

  return time_domain, freq_domain
#create a new nested dic "DATA" with all subjects
DATA = {}
DATA['TIME'] = {} #time-domain data
DATA['FREQ'] = {} #freq-domain data

for p in range(participants_num): # number of participants = 15
    DATA['TIME']['subject'+str(p)] = {}
    DATA['FREQ']['subject'+str(p)] = {}
    for c in range(num_cond):  # number of conditions  = 7
        DATA['TIME']['subject'+str(p)]['condition'+str(c)], DATA['FREQ']['subject'+str(p)]['condition'+str(c)] = Analysis(HCPS['subject'+str(p)]['condition'+str(c)],trialnum[c])

C:\Users\hachou\AppData\Local\Temp/ipykernel_7648/2600246000.py:8: RuntimeWarning: divide by zero encountered in true_divide
  M = lambda s : 1/(s**2 + s) #2nd order machine
C:\Users\hachou\AppData\Local\Temp/ipykernel_7648/2600246000.py:8: RuntimeWarning: invalid value encountered in true_divide
  M = lambda s : 1/(s**2 + s) #2nd order machine
C:\Users\hachou\AppData\Local\Temp/ipykernel_7648/2600246000.py:66: RuntimeWarning: invalid value encountered in true_divide
  Wr_h.append( r_h[i] / M_h )
C:\Users\hachou\AppData\Local\Temp/ipykernel_7648/2600246000.py:82: RuntimeWarning: invalid value encountered in multiply
  Md_h.append( d_h[i] * M_h )


## non stim user inputs

In [4]:
#mask for index of freq that are less than 1Hz
def lessthanone(x): return x <= 1.0
indexone = [idx for idx, element in enumerate(xf) if lessthanone(element)]
unwanted_index = {4,  6, 10, 14, 22, 26, 34, 38}
nonstimulated_index = [i for i in indexone if i not in unwanted_index]


In [5]:
for p in range(participants_num): # number of participants = 15
    for c in range(num_cond):  # number of conditions  = 7
        DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['UX'] = [] #non-stim u in freq domain (freq less than 1)
        DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U0X'] = []
        DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U1X'] = []

        for i in range(trialnum[c]):
            DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['UX'].append( DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U'][i][nonstimulated_index] )
            DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U0X'].append( DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U0'][i][nonstimulated_index] )
            DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U1X'].append( DATA['FREQ']['subject'+str(p)]['condition'+str(c)]['U1'][i][nonstimulated_index] )


In [6]:
DATA['FREQ']['subject'+str(p)]['condition'+str(c)].keys()

dict_keys(['MD', 'R', 'Y', 'D', 'WR', 'U', 'U0', 'U1', 'UX', 'U0X', 'U1X'])

## save DATA

In [7]:
import pickle
b_file = open("DATA.pkl", "wb")
pickle.dump(DATA, b_file)
b_file.close()